## Part 2: VacationPy

In [119]:
 # Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from config import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)

In [120]:
# Read Csv
city_weather = pd.read_csv("../output_data/cities.csv")
city_weather.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Clouds,Humidity,Wind Speed
0,0,Norman Wells,CA,1620875887,65.2820,-126.8329,40.28,90,81,2.30
1,1,Muroto-misakicho,JP,1620875944,33.2833,134.1500,71.56,100,84,10.85
2,2,Saskylakh,RU,1620875845,71.9167,114.0833,32.95,100,91,6.85
3,3,New Norfolk,AU,1620875861,-42.7826,147.0587,61.45,100,79,5.01
4,4,Cape Town,ZA,1620875575,-33.9258,18.4232,54.86,20,84,2.77


## Create a heatmap that displays the humidity for every city from Part 1

In [121]:
#Use the Latitude and Longitude as locations
locations = city_weather[["Lat","Lng"]]

# Fill NaN values and convert to float
humid = city_weather["Humidity"]



In [125]:
# Plot Heatmap
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'margin': '0 auto 0 auto'
}

#Plot heatmap
figure = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

#Create heat layer
heatlayer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

figure.add_layer(heatlayer)


#save fig to file
plt.savefig("Images/heatmap.png")

figure

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', width='1000px'))

<Figure size 432x288 with 0 Axes>

## Narrow down the DF to find ideal weather conditions

In [126]:
#Narrow down the cities to find ideal weather condistion. For example: 
#A max temperature lower than 80 but higher than 70
#Wind speed less than 10 mph
#Zero cloudiness
#Drop the rows that don't contain all three conditions
narrowdown_df = city_weather.loc[(city_weather["Max Temp"] > 70) & (city_weather["Max Temp"] < 80)& (city_weather["Wind Speed"] < 10) & (city_weather["Clouds"]== 0)].dropna()
narrowdown_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Clouds,Humidity,Wind Speed
105,105,Catalina,US,1620876237,32.5056,-110.9211,76.82,0,11,7.20
107,107,Aswān,EG,1620876237,24.0934,32.9070,76.28,0,18,7.56
125,125,Assiut,EG,1620875951,27.1810,31.1837,71.98,0,23,4.43
129,129,Port Macquarie,AU,1620875671,-31.4333,152.9167,72.88,0,64,8.90
309,309,Mossamedes,AO,1620875934,-15.1961,12.1522,73.17,0,66,2.91


## Using Google Place API to find the first hotel for each city located within 5000 meters of coodrdinates

In [127]:
#Store into variable 
hotel = narrowdown_df.loc[:,["City","Country","Lat","Lng"]]

#Add a column of "Hotel Name" to the DataFrame
hotel["Hotel Name"] = ""
hotel.head()

,City,Country,Lat,Lng,Hotel Name
105,Catalina,US,32.5056,-110.9211,
107,Aswān,EG,24.0934,32.9070,
125,Assiut,EG,27.1810,31.1837,
129,Port Macquarie,AU,-31.4333,152.9167,
309,Mossamedes,AO,-15.1961,12.1522,


In [128]:
#Set parameters to find hotel for each city located within 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
          "radius" : 5000,
          "type" : "lodging",
          "keyword" : "Hotel",
          "key" : g_key
}
     
    

In [133]:
# Use the latituda /longitude to identify airports
for index, row in hotel.iterrows():
    # get lat, lng , city name from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng},{city_name}"
    
    #Print Url
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Skipping.")

Retrieving Results for Index 105: Catalina.
Skipping.
Retrieving Results for Index 107: Aswān.
Skipping.
Retrieving Results for Index 125: Assiut.
Skipping.
Retrieving Results for Index 129: Port Macquarie.
Skipping.
Retrieving Results for Index 309: Mossamedes.
Skipping.
Retrieving Results for Index 431: Kiama.
Skipping.
Retrieving Results for Index 446: Vila Velha.
Skipping.
Retrieving Results for Index 510: Seoul.
Skipping.


In [134]:
#Display the dataframe
hotel.head()

,City,Country,Lat,Lng,Hotel Name
105,Catalina,US,32.5056,-110.9211,
107,Aswān,EG,24.0934,32.9070,
125,Assiut,EG,27.1810,31.1837,
129,Port Macquarie,AU,-31.4333,152.9167,
309,Mossamedes,AO,-15.1961,12.1522,


In [135]:
# Using the template to plot the hotels 
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]
locations = hotel[["Lat", "Lng"]]

In [136]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
figure2 = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Plot the hotels on top of the humidity heatmap
hotellayer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel.iterrows()]
)

# Add heat layer and 
figure2.add_layer(heatlayer)
figure2.add_layer(hotellayer)

#save fig to file
plt.savefig("Images/hotel_map.png")

# Display figure
figure2

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

<Figure size 432x288 with 0 Axes>